# Dealing with FreeRiders in P2P 

- Freriders can be present in any p2p system on multiple levels. 
- We address the freeriders on the networking level, i.e. when peer don't particicpate in the protocol to save the resources. 
    - Active freeriding on the networking level - trying to decrease the quality of service from other peers and increase own QoS. 
    


## Experiment 1

**Context**: Exchanging the transactions and data, such that every peer receives it. 

**Examples**: Bitcoin, Gossip Network, DHT 


### Process and plan 

- Create an overlay with peer spreaded either random:
    * Unstructured: Gossip Network like in Bitcoin. 
    * Structured: DHT-like network with a load. Wikiedia on IPFS - patterns of the access.  
- Load on the peers: 
    * Unstructured: The load is generated from the clients. Clients produce transactions by Pareto law. Each peer must spread the transactions evenly.  
    * Structured: clients generate load by requesting files. Peer store the files unevenly but relay the traffic. Search in DHT using the Zipf law distribution 
    
    
#### Attack scenarios 

- Model one freerider: See the effect of the freeriding on one peer. How much you can save while not losing the priority;
- Model K freeriders: What is the effect on the whole system. How it will change with the number of freeriders


In [ ]:
from random import choice

import matplotlib.pyplot as plt
import networkx as nx

# Define locations
from ledger0.application_handler import AccountableGossip, AccountableMessageProducer
from ledger0.crawler import CrawlerService
from ledger0.ledger import LedgerNetterWorker
from p2psimpy.config import *
from p2psimpy.consts import *
from p2psimpy.services.connection_manager import BaseConnectionManager

# Locations and latencies between the points 
class Locations(Config):
    locations = ['Ohio', 'Ireland', 'Tokyo']
    latencies = {
        'Ohio': {'Ohio': Dist('invgamma', (5.54090, 0.333305, 0.987249)),
                 'Ireland': Dist('norm', (73.6995, 1.19583092197097127)),
                 'Tokyo': Dist('norm', (156.00904977375566, 0.09469886668079797))
                },
        'Ireland':{'Ireland': Dist('invgamma', (6.4360455224301525, 0.8312748033308526, 1.086191852963273)),
                   'Tokyo': Dist('norm', (131.0275, 0.25834811785650774))
                  },
        'Tokyo': {'Tokyo':  Dist('invgamma', (11.104508341331055, 0.3371934865734555, 2.0258998705983737))}
    }
    
# Peer physical properties 
class PeerConfig(Config):
    location = Dist('sample', Locations.locations)
    bandwidth_ul = Dist( 'norm', (50*MBit, 10*MBit))
    bandwidth_dl = Dist( 'norm', (50*MBit, 10*MBit))

# Configuration used for our GossipService
class GossipConfig(Config):
    exclude_types={'client',}

# We have not two types of nodes: *peer* and *client*
def prepare_peer_types():
    return { 'peer': PeerType(PeerConfig,   {BaseConnectionManager: None,
                                             AccountableGossip: GossipConfig,
                                             LedgerNetterWorker: None,
                                             CrawlerService: None
                                             }),
             'client': PeerType(PeerConfig, (BaseConnectionManager,
                                             AccountableMessageProducer,
                                             LedgerNetterWorker,
                                             CrawlerService
                                             ))}

def prepare_topology(num_peers=25, num_clients=1):    
    # Create network topology
    G = nx.erdos_renyi_graph(num_peers, 0.4)   
    nx.relabel_nodes(G, {k: k+1 for k in G.nodes()} ,copy=False)
    
    # Connect the client node to a random peer
    client_edges = [(i, choice(list(G.nodes())))
                    for i in range(num_peers+1, num_clients+num_peers+1)]
    G.add_edges_from(client_edges)

    types_map = {k: 'peer' if k < num_peers+1 else 'client' for k in G.nodes()}
    # Assign a peer type to the peers 
    nx.set_node_attributes(G, types_map , 'type')
    return G

def visualize_peer_client_network(G):
    plt.figure(figsize=(10,10))

    # Draw client/ peer network 

    master_nodes = [n for (n,ty) in \
        nx.get_node_attributes(G,'type').items() if ty == 'peer']
    client_nodes = [n for (n,ty) in \
        nx.get_node_attributes(G,'type').items() if ty == 'client']

    pos = nx.kamada_kawai_layout(G)

    nx.draw_networkx_nodes(G, pos, nodelist=master_nodes,
                           node_color='blue', node_shape='o',
                           node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=client_nodes,
                           node_color='green', node_shape='^',
                           node_size=100, label=1)

    nx.draw_networkx_labels(G, pos, labels={k:k for k in master_nodes},
                            font_color='w')

    nx.draw_networkx_edges(G, pos,
                           edgelist=G.subgraph(master_nodes).edges(),
                           width=1.5)
    nx.draw_networkx_edges(G, pos,
                           edgelist=G.edges(nbunch=client_nodes),
                           style='dotted')


In [ ]:
G =  prepare_topology()
visualize_peer_client_network(G)

In [ ]:
from p2psimpy.simulation import BaseSimulation

#  logger_dir='new_log', enable_logger=True
sim = BaseSimulation(Locations, G, prepare_peer_types())
sim.run(50_000)

In [ ]:
sim.peers[1].counters.receive_counters

In [ ]:
sim.peers[13].counters.send_counters

In [ ]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

In [ ]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

In [ ]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

In [ ]:
print(sim.peers[1].storage['work_graph'].G.number_of_edges())
print(sim.peers[2].storage['work_graph'].G.number_of_edges())
print(sim.peers[3].storage['work_graph'].G.number_of_edges())
print(sim.peers[4].storage['work_graph'].G.number_of_edges())
print(sim.peers[5].storage['work_graph'].G.number_of_edges())

In [ ]:
G.number_of_edges()


### How quick we converge

*Convergance - $\delta t$ to have $|E|$ nodes*

- Random walk on ego network + Random 5 transactions on response + $500 ms$:
  $50 s$, 100 rounds of crawling
-


### Overlay enhancements
1. **An accounting system**

Count the messages bandwidth between the peers based on pairwise interaction.
The peers exchange the counters and store their views together
with signature.
2. **Sampling algorithm**

Sample the transactions from the network starting crawling the data
from ego work graph. Sampling the transactions


3. Reputation (Classification) algorithm
4. Scheduling, Routing advices and Restructuring
---
- Handler to store the messages - with a graph structure
- MessageProducer - interaction with a peer
- Sampling - a runner to collect and put in the storage.
- Reputation - a runner, periodically write a reputation scores.
- Routing advices - handler for routing. A smart router.
